# Normalized Geo Match™️
Developed by *Bas Baudoin* for *[Happy Horizon](https://happyhorizon.com/)™️*


[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/10hu9TqJztZTSwTKOfhE5foRPEL3BY38B#scrollTo=OrLmr59U4lve)

- Assignment values: **`1 = treatment`** (B) and  **`2 = control`** (A)
- Checkmark chapters (✔️) are for manual validation. Easily take a look if all data still looks okay after transformations
- Input dataset format:

| date       | geo | response   | cost    | pair | assignment | Region       |
|------------|-----|------------|---------|------|------------|--------------|
| 2023-12-01 | 3.0 | 374.779999 | 0.0     | 3.0  | 2.0        | Friesland    |
| 2023-12-01 | 1.0 | 76.42      | 0.84    | 2.0  | 1.0        | Drenthe      |
| 2023-12-01 | 8.0 | 4432.929997| 0.0     | 5.0  | 2.0        | North Holland|
| 2023-12-01 | 4.0 | 972.620001 | 0.0     | 4.0  | 2.0        | Gelderland   |
| 2023-12-01 | 10.0| 2469.579999| 12.42   | 5.0  | 1.0        | South Holland|
| 2023-12-01 | 11.0| 1214.31    | 4.690252| 4.0  | 1.0        | Utrecht      |
| 2023-12-01 | 6.0 | 69.96      | 0.0     | 2.0  | 2.0        | Limburg      |
| 2023-12-01 | 5.0 | 160.19     | 4.39    | 3.0  | 1.0        | Groningen    |
| 2023-12-01 | 2.0 | 174.31     | 0.0     | 1.0  | 2.0        | Flevoland    |
| 2023-12-02 | 8.0 | 2404.19    | 0.0     | 5.0  | 2.0        | North Holland|


## 📦 Imports

In [ ]:
import pandas as pd
import re

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

## ⚙️ Settings

In [ ]:
'''
💡  Recommendation: save this config file locally for future use, along with the csv file

    # Add names for reference:
    Client name: clientX
    Test name: upper funnel NL 2024
    Description: containing data halfway, no cooldown
'''
config = {
    'design_eval_start_date': '2024-01-01',
    'design_eval_end_date': '2024-03-01',

    'test_start_date': '2024-03-10',
    'test_end_date': '2024-05-26',

    'cooldown_end_date': '2024-05-27', # Only used for visuals

    'pairs_include': '1,2', # Select pairs e.g '1,2,3'

    'file_name': 'upperf_2024-05-28_revenue.csv',
    'response_name': 'Revenue', # E.g. 'Revenue' or 'Sessions'
}

## 🪟 Prepare dataframe

In [ ]:
# Convert config dates
test_start_date = pd.to_datetime(config['test_start_date'])
test_end_date = pd.to_datetime(config['test_end_date'])
cooldown_end_date = pd.to_datetime(config['cooldown_end_date'])
design_eval_start_date = pd.to_datetime(config['design_eval_start_date'])
design_eval_end_date = pd.to_datetime(config['design_eval_end_date'])

# Prepare and check initial dataframe
data = pd.read_csv(config['file_name'])

# Basic type transformations
data['date'] = pd.to_datetime(data['date'])

# Filter the data to include only rows after the design start date
data = data[data['date'] >= design_eval_start_date]

# Convert numeric columns
for colname in ['geo', 'pair', 'assignment', 'response', 'cost']:
  data[colname] = pd.to_numeric(data[colname])

# Convert pairs string to array
pairs_include = [] if config['pairs_include'] == '' else [
    int(re.sub(r'\W+', '', x)) for x in config['pairs_include'].split(',')
]

# Keep only used pairs
data = data[data['pair'].isin(pairs_include)]

# Preview
data.head()

### (optional) Switch assignment
💡 Check if your treatment group = 1 and control = 2, otherwise run the code below to switch it

In [ ]:
# ⚙️ Optional switch assignment, for debugging purposes
SWITCH_ASSIGNMENT = True # Default False
if (SWITCH_ASSIGNMENT):
    data['assignment'] = data['assignment'].map({1: 2, 2: 1})

## ✔️ (optional) Validate pairs

Manually check the table if everything looks valid.

In [ ]:
# ✔️ CHECK: Overview of used regions
validate_grouped_df = data.groupby(['Region', 'pair']).agg({'response': 'sum', 'cost': 'sum', 'assignment': 'first'}).reset_index()
validate_grouped_df.sort_values(by='pair')

## 🪟 Create normalized dataframe

The goal is to align pairs so we can calculate the incrementality as precisely as possible. There are several methods to do this, but I found the normalization based on each timeseries' maximum value the most robust. If your data contains outliers, you might want to use the smooth max scaling method. An alternate method is to align the timeseries based on the test start date. I suggest trying out different methods and look at the charts in the following steps what seems to align best for your data.

Run one of the four options.

### Option 1: max scaling

In [ ]:
# Ensure the dates in the config are converted to datetime if not already
design_eval_start_date = pd.to_datetime(config['design_eval_start_date'])
design_eval_end_date = pd.to_datetime(config['design_eval_end_date'])

# Assuming 'data' is your DataFrame and it has a datetime column named 'date'
norm_df = data.copy()

def normalize_and_get_multipliers(norm_df, column, start_date, end_date):
    # Filter the DataFrame for the design evaluation period
    period_df = norm_df[(norm_df['date'] >= start_date) & (norm_df['date'] <= end_date)]

    # Group by 'geo' and 'pair', and calculate the max value within the specified period
    grouped = period_df.groupby(['geo', 'pair'])[column]
    max_vals = grouped.transform('max')

    # Use the max values from the filtered period to normalize the entire dataset
    normalized_values = norm_df[column] / norm_df.groupby(['geo', 'pair'])[column].transform(lambda x: max_vals.reindex(x.index, method='ffill'))

    return normalized_values, max_vals

# Apply normalization to 'cost' and 'response' columns for each region using the design evaluation period
norm_df['cost_norm'], norm_df['cost_max'] = normalize_and_get_multipliers(norm_df, 'cost', design_eval_start_date, design_eval_end_date)
norm_df['response_norm'], norm_df['response_max'] = normalize_and_get_multipliers(norm_df, 'response', design_eval_start_date, design_eval_end_date)

norm_df

### Option 2: smooth max scaling

In [ ]:
norm_df = data.copy()

def normalize_and_get_multipliers(norm_df, column):
    # Apply 7-day rolling average and then group by 'geo' and 'pair'
    rolled = norm_df.groupby(['geo', 'pair'])[column].rolling(window=7, min_periods=1).mean().reset_index(level=[0,1], drop=True)
    norm_df[f'{column}_rolled'] = rolled

    # Group by 'geo' and 'pair' again to find the max of the rolling averages
    max_vals = norm_df.groupby(['geo', 'pair'])[f'{column}_rolled'].transform('max')

    # Normalize original column values by these max values
    normalized_values = norm_df[column] / max_vals
    return normalized_values, max_vals

# Apply normalization to 'cost' and 'response' columns for each region
norm_df["cost_norm"], norm_df["cost_max"] = normalize_and_get_multipliers(norm_df, "cost")
norm_df["response_norm"], norm_df["response_max"] = normalize_and_get_multipliers(norm_df, "response")

norm_df

### Option 3: start date scaling

In [ ]:
# ! Error when cost or response = 0 at test_start_date

norm_df = data.copy()

def normalize_based_on_date(norm_df, column, date):
    start_date_mask = norm_df['date'] == date

    # Group by 'geo' and 'pair', then apply the normalization
    def normalize(group):
        start_value = group.loc[start_date_mask & (group['geo'] == group.name[0]) & (group['pair'] == group.name[1]), column].iloc[0]
        return group[column] / start_value

    normalized_values = norm_df.groupby(['geo', 'pair']).apply(normalize).reset_index(level=[0,1], drop=True)
    return normalized_values.fillna(0)

# Apply the new normalization to 'cost' and 'response' columns
norm_df['cost_norm'] = normalize_based_on_date(norm_df, 'cost', config['test_start_date'])
norm_df['response_norm'] = normalize_based_on_date(norm_df, 'response', config['test_start_date'])

norm_df

### Option 4: smooth start date scaling

In [ ]:
norm_df = data.copy()

def normalize_based_on_date(norm_df, column, date):
    # Calculate 7-day rolling average for the column
    rolled = norm_df.groupby(['geo', 'pair'])[column].rolling(window=7, min_periods=1).mean().reset_index(level=[0,1], drop=True)
    norm_df[f'{column}_rolled'] = rolled

    # Create a mask for the start date
    start_date_mask = norm_df['date'] == date

    # Group by 'geo' and 'pair', then apply the normalization
    def normalize(group):
        start_value = group.loc[start_date_mask & (group['geo'] == group.name[0]) & (group['pair'] == group.name[1]), f'{column}_rolled'].iloc[0]
        return group[column] / start_value

    normalized_values = norm_df.groupby(['geo', 'pair']).apply(normalize).reset_index(level=[0,1], drop=True)
    return normalized_values.fillna(0)

# Apply the new normalization to 'cost' and 'response' columns
norm_df['cost_norm'] = normalize_based_on_date(norm_df, 'cost', config['test_start_date'])
norm_df['response_norm'] = normalize_based_on_date(norm_df, 'response', config['test_start_date'])

norm_df

## 🪟 Create control vs. treatment pairs dataframe

In order to be able to compare pairs, a new dataframe has to be created where the data of each of the pairs is on the same row.

In [ ]:
pairs_df = norm_df.copy()

# Type modification
pairs_df['pair'] = pairs_df['pair'].astype(int)
pairs_df['assignment'] = pairs_df['assignment'].astype(int)
pairs_df['pair'] = pairs_df['pair'].astype(str)
pairs_df['assignment'] = pairs_df['assignment'].astype(str)

# Pivot the table and create column names
pairs_df = pairs_df.pivot_table(index=['date', 'pair'], columns='assignment', values=['geo', 'response', 'cost', 'Region', 'cost_norm', 'response_norm'], aggfunc='first')

# Create readable names for new columns
pairs_df.columns = ['_'.join(map(str, col)).strip() for col in pairs_df.columns.values]
rename_dict = {
    '_1': '_treatment',
    '_2': '_control'
}
for old_suffix, new_suffix in rename_dict.items():
    pairs_df.columns = pairs_df.columns.str.replace(old_suffix, new_suffix)

pairs_df.reset_index(inplace=True)

pairs_df.head()

## ✔️ Charting

This is the first main output of the analysis, as a final result and for inspection. Check for the following things:

- [ ] Do the pairs align in the test periods?
- [ ] Are the interventions visible? For example when pausing one of the pairs in the experiment, the cost line should drop to 0 at the test date.
- [ ] Do the pair lines align sufficiently at the start dates?

If the data doesn't align well, you can either choose another scaling option and/or choose to leave out certain pairs.

In [ ]:
df_plot = pairs_df.copy()

# ⚙️ Option to smootth the results
SMOOTH_RESULTS = False
ROLLING_DAYS = 4 # Default 4 if enabled, more days = smoother

if (SMOOTH_RESULTS):
    df_plot['response_norm_treatment'] = df_plot['response_norm_treatment'].rolling(ROLLING_DAYS).mean()
    df_plot['response_norm_control'] = df_plot['response_norm_control'].rolling(ROLLING_DAYS).mean()
    df_plot['cost_norm_treatment'] = df_plot['cost_norm_treatment'].rolling(ROLLING_DAYS).mean()
    df_plot['cost_norm_control'] = df_plot['cost_norm_control'].rolling(ROLLING_DAYS).mean()

# Plotting the data
unique_pairs = df_plot['pair'].unique()
for pair in unique_pairs:
    df_subset = df_plot[df_plot['pair'] == pair]

    # Extracting region labels for the current pair
    region_treatment = df_subset['Region_treatment'].iloc[0]
    region_control = df_subset['Region_control'].iloc[0]

    # Plot for response values
    plt.figure(figsize=(19, 3))
    plt.gca().set_facecolor('#FFF9C4')  # Yellow background for response charts
    plt.plot(df_subset['date'], df_subset['response_norm_treatment'], color='red', label=f'{config["response_name"]} Treatment ({region_treatment})')
    plt.plot(df_subset['date'], df_subset['response_norm_control'], color='gray', label=f'{config["response_name"]} Control ({region_control})')

    # Adding vertical lines
    plt.axvline(test_start_date, color='green', linestyle='--', label='Test Start')
    plt.axvline(test_end_date, color='red', linestyle='--', label='Test End')
    plt.axvline(cooldown_end_date, color='blue', linestyle='--', label='Cooldown End')
    plt.axvline(design_eval_start_date, color='purple', linestyle='--', label='Design Eval Start')
    plt.axvline(design_eval_end_date, color='orange', linestyle='--', label='Design Eval End')

    # Setting legend on the left side
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.title(f'{config["response_name"]} for Pair {pair}')
    plt.xlabel('Date')
    plt.ylabel(f'{config["response_name"]} Values')
    plt.grid(True)
    plt.show()

    # Plot for cost values
    plt.figure(figsize=(19, 3))
    plt.gca().set_facecolor('#E3F2FD')  # Blue background for cost charts
    plt.plot(df_subset['date'], df_subset['cost_norm_treatment'], color='red', label=f'Cost Treatment ({region_treatment})')
    plt.plot(df_subset['date'], df_subset['cost_norm_control'], color='gray', label=f'Cost Control ({region_control})')

    # Adding vertical lines
    plt.axvline(test_start_date, color='green', linestyle='--', label='Test Start')
    plt.axvline(test_end_date, color='red', linestyle='--', label='Test End')
    plt.axvline(cooldown_end_date, color='blue', linestyle='--', label='Cooldown End')
    plt.axvline(design_eval_start_date, color='purple', linestyle='--', label='Design Eval Start')
    plt.axvline(design_eval_end_date, color='orange', linestyle='--', label='Design Eval End')

    # Setting legend on the left side
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.title(f'Cost for Pair {pair}')
    plt.xlabel('Date')
    plt.ylabel('Cost Values')
    plt.grid(True)
    plt.show()
    print('--------')

## ✔️ (optional) Charting - with difference colors [experimental]

This is an alternative method to create the chart, where the difference (incrementality) is colored. This gives a better indication of the amount of incrementality, but looks a bit messier.

In [ ]:
df_plot = pairs_df.copy()

# ⚙️ Option to smootth the results
SMOOTH_RESULTS = False
ROLLING_DAYS = 4 # Default 4 if enabled, more days = smoother

if (SMOOTH_RESULTS):
    df_plot['response_norm_treatment'] = df_plot['response_norm_treatment'].rolling(ROLLING_DAYS).mean()
    df_plot['response_norm_control'] = df_plot['response_norm_control'].rolling(ROLLING_DAYS).mean()
    df_plot['cost_norm_treatment'] = df_plot['cost_norm_treatment'].rolling(ROLLING_DAYS).mean()
    df_plot['cost_norm_control'] = df_plot['cost_norm_control'].rolling(ROLLING_DAYS).mean()
print("Blue area = incremental, red area = anti incremental")

# Plotting the data
unique_pairs = df_plot['pair'].unique()
for pair in unique_pairs:
    df_subset = df_plot[df_plot['pair'] == pair]

    # Extracting region labels for the current pair
    region_treatment = df_subset['Region_treatment'].iloc[0]
    region_control = df_subset['Region_control'].iloc[0]

    # Plot for response values
    plt.figure(figsize=(19, 3))
    plt.gca().set_facecolor('#FFF9C4')
    plt.plot(df_subset['date'], df_subset['response_norm_treatment'], color='red', label=f'Response Treatment ({region_treatment})')
    plt.plot(df_subset['date'], df_subset['response_norm_control'], color='gray', label=f'Response Control ({region_control})')

    # Filling the area between the lines
    plt.fill_between(df_subset['date'], df_subset['response_norm_treatment'], df_subset['response_norm_control'],
                     where=(df_subset['response_norm_treatment'] > df_subset['response_norm_control']),
                     facecolor='red', alpha=0.3, interpolate=True)

    plt.fill_between(df_subset['date'], df_subset['response_norm_treatment'], df_subset['response_norm_control'],
                     where=(df_subset['response_norm_treatment'] <= df_subset['response_norm_control']),
                     facecolor='blue', alpha=0.3, interpolate=True)

    # Adding vertical lines
    plt.axvline(test_start_date, color='red', linestyle='--', label='Test Start')
    plt.axvline(test_end_date, color='red', linestyle='--', label='Test End')
    plt.axvline(cooldown_end_date, color='blue', linestyle='--', label='Cooldown End')
    plt.axvline(design_eval_start_date, color='purple', linestyle='--', label='Design Eval Start')
    plt.axvline(design_eval_end_date, color='orange', linestyle='--', label='Design Eval End')

    # Setting legend on the left side
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.title(f'Response by Date for Pair {pair}')
    plt.xlabel('Date')
    plt.ylabel('Response Values')
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(19, 3))
    plt.gca().set_facecolor('#E3F2FD')
    plt.plot(df_subset['date'], df_subset['cost_norm_treatment'], color='red', label=f'Cost Treatment ({region_treatment})')
    plt.plot(df_subset['date'], df_subset['cost_norm_control'], color='gray', label=f'Cost Control ({region_control})')

    # Filling the area between the lines for cost
    plt.fill_between(df_subset['date'], df_subset['cost_norm_treatment'], df_subset['cost_norm_control'],
                     where=(df_subset['cost_norm_treatment'] > df_subset['cost_norm_control']),
                     facecolor='red', alpha=0.3, interpolate=True)

    plt.fill_between(df_subset['date'], df_subset['cost_norm_treatment'], df_subset['cost_norm_control'],
                     where=(df_subset['cost_norm_treatment'] <= df_subset['cost_norm_control']),
                     facecolor='blue', alpha=0.3, interpolate=True)

    # Adding vertical lines
    plt.axvline(test_start_date, color='red', linestyle='--', label='Test Start')
    plt.axvline(test_end_date, color='red', linestyle='--', label='Test End')
    plt.axvline(cooldown_end_date, color='blue', linestyle='--', label='Cooldown End')
    plt.axvline(design_eval_start_date, color='purple', linestyle='--', label='Design Eval Start')
    plt.axvline(design_eval_end_date, color='orange', linestyle='--', label='Design Eval End')

    # Setting legend on the left side
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.title(f'Cost by Date for Pair {pair}')
    plt.xlabel('Date')
    plt.ylabel('Cost Values')
    plt.grid(True)
    plt.show()
    print('--------')

## ✔️ (optional) Statistical validations

Useful to compare different settings. Run all these cells first to get the incrementality values at the end.

- Here you need high correlations and low MSE
- When using `max` or `smooth_max` it's best to choose the model with lowest MSE total

In [ ]:
# Prepare function for metrics
corr_df = pairs_df.copy()

def calculate_metrics_for_period(group, start_date, end_date):
    metrics = {}
    # Filter the group for the specified period
    period_data = group[(group['date'] >= start_date) & (group['date'] <= end_date)]

    for col in ['cost', 'response']:
        treatment_col = f'{col}_norm_treatment'
        control_col = f'{col}_norm_control'

        corr = period_data[treatment_col].corr(period_data[control_col])
        metrics[f'{col}_pcorr'] = corr

        mse = mean_squared_error(period_data[treatment_col], period_data[control_col])
        metrics[f'{col}_mse'] = mse

    return pd.Series(metrics)

design_period_results = corr_df.groupby('pair').apply(calculate_metrics_for_period, design_eval_start_date, design_eval_end_date).reset_index()
test_period_results = corr_df.groupby('pair').apply(calculate_metrics_for_period, test_start_date, test_end_date).reset_index()

total_cost_mse = design_period_results['cost_mse'].sum()
print("Total of cost_mse     :", total_cost_mse)

total_response_mse = design_period_results['response_mse'].sum()
print("Total of response_mse :", total_response_mse)

## ➖ Incrementality values

In [ ]:
from IPython.display import Markdown, display

def printm(c): # Utility function
    return display(Markdown((c)))

C_S = '' # ⚙️ Currency symbol, leave empty '' or curency with space '€ '
diff_df = pairs_df.copy()

# Calculate total differences
diff_df['cost_norm_diff'] = diff_df['cost_norm_treatment'] - diff_df['cost_norm_control']
diff_df['response_norm_diff'] = diff_df['response_norm_treatment'] - diff_df['response_norm_control']

# # Filter and denormalize! So that's the absolute difference from the normalized control baseline
test_period_df = diff_df[(diff_df['date'] >= test_start_date) & (diff_df['date'] <= test_end_date)].copy()
test_period_df['response_abs_diff'] = test_period_df['response_norm_diff'] * test_period_df['response_treatment']
test_period_df

grouped = test_period_df.groupby('pair')
results = {}

for pair_id, group in grouped:
    test_period_group = group[(group['date'] >= test_start_date) & (group['date'] <= test_end_date)]

    norm_cost_diff = test_period_group['cost_norm_diff'].mean() # not used
    norm_response_diff = test_period_group['response_norm_diff'].mean()
    abs_response_diff = test_period_group['response_abs_diff'].sum()
    abs_response_diff_avg = test_period_group['response_abs_diff'].mean()
    abs_response_avg = test_period_group['response_treatment'].mean()

    region_treatment = group['Region_treatment'].iloc[0]
    region_control = group['Region_control'].iloc[0]

    # Display
    printm(f'''### Pair {pair_id}: {region_control} (control) - {region_treatment} (treatment)

Effect in test period: {C_S}{round(abs_response_diff)}

Relative effect: {round(norm_response_diff * 100, 1)}%''')